# Generating images with MX-Font model from a reference style
In this example we'll generate images with trained MX-Font model from a reference style.
If you want to generate multiple styles, please check using `eval.py` instead of using this example file (because it is much simpler to load the referece styles).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 1. Loading packages
* First, load the packages used in this code.
* All of the packages are avilable in `pip`.

In [ ]:
!pip install sconf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 9.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
%cd "/content/drive/MyDrive/2023학교/AIKU/2023_2_project2/mxfont-main"

/content/drive/MyDrive/2023학교/AIKU/2023_2_project2/mxfont-main


In [ ]:
!pip install numpy scipy scikit-image tqdm jsonlib-python3 fonttools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for jsonlib-python3: filename=jsonlib_python3-1.6.1-cp310-cp310-linux_x86_64.whl size=72615 sha256=57b8c7d66e889145100a200e8260ce5b239c6fc5d308de5e28e79a64052335d2
  Stored in directory: /root/.cache/pip/wheels/1f/48/2b/3b4e8a617b152bacc255f98c81f38f099b1ac06209d6c48d8a
Successfully built jsonlib-python3


In [ ]:
import json
from pathlib import Path
from PIL import Image

import torch
from sconf import Config
from torchvision import transforms

* These modules are defined in this repository.

In [ ]:
import sys
sys.path.append("/content/drive/My Drive/mxfont/mxfont")

In [ ]:
import models
from datasets import read_font, render
from utils import save_tensor_to_image

### 2. Build model
* Build and load the trained model.
* `weight_path` :
    * The location of the trained model weight.

In [ ]:
import torch

print(torch.cuda.is_available())

True


In [ ]:
!pwd

/content/drive/MyDrive/2023학교/AIKU/2023_2_project2/mxfont-main


In [ ]:
########################################################
weight_path = "/content/drive/MyDrive/2023학교/AIKU/2023_2_project2/mxfont-main/train_results/2023_12_24_09:54:18/checkpoints/018000.pth"
#weight_path = "/content/drive/MyDrive/2023학교/AIKU/2023_2_project2/mxfont-main/train_results/2023_11_30_08:27:41/checkpoints/027000.pth"  # path to weight to infer
#weight_path = "/content/drive/MyDrive/2023학교/AIKU/2023_2_project2/mxfont-main/train_results/2023_11_29_21:36:41/last.pth"
#weight_path = "/content/drive/MyDrive/2023학교/AIKU/2023_2_project2/mxfont-main/generator.pth"
########################################################

cfg = Config("/content/drive/MyDrive/2023학교/AIKU/2023_2_project2/mxfont-main/cfgs/eval.yaml", default="/content/drive/MyDrive/2023학교/AIKU/2023_2_project2/mxfont-main/cfgs/defaults.yaml")
transform = transforms.Compose(
    [transforms.Grayscale(num_output_channels=1),transforms.Resize((128, 128)), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
)
decomposition = json.load(open("/content/drive/MyDrive/fewshot-font-generation-main/data/kor/decomposition.json"))

g_kwargs = cfg.get('g_args', {})
gen = models.Generator(1, cfg.C, 1, **g_kwargs).cuda().eval()
weight = torch.load(weight_path)
if "generator_ema" in weight:
    weight = weight["generator_ema"]
gen.load_state_dict(weight)

<All keys matched successfully>

### 3. Load reference images.
* `ref_path`:
    * The path of reference font or images.
    * If you are using a ttf file, set this to the location of the ttf file.
    * If you want to use rendered images, set this to the path to the directory which contains the reference images.
* `extension`:
    * If you are using image files, set this to their extension(png, jpg, etc..).
    * This will be ignored if `use_ttf` is True.
* `batch_size`:
    * The number of images inferred at once.

In [ ]:
import pdb

In [ ]:
########################################################
ref_path = "/content/drive/MyDrive/2023학교/AIKU/2023_2_project2/mxfont-main/data/images/test/MaShanZheng-Regular"  # Path to the reference images
extension = "png"  # Extension of the reference images
batch_size = 3  # The batch size
########################################################

ref_paths = Path(ref_path).glob(f"*.{extension}")
#temp_img = [transform(Image.open(str(p))) for p in ref_paths]
#pdb.set_trace()
ref_imgs = torch.stack([transform(Image.open(str(p))) for p in ref_paths]).cuda()
ref_batches = torch.split(ref_imgs, batch_size)

### 4. Extract style factors from reference images.

In [ ]:
style_facts = {}
import numpy as np
for batch in ref_batches:
    style_fact = gen.factorize(gen.encode(batch), 0)
    for k in style_fact:
        style_facts.setdefault(k, []).append(style_fact[k])
style_facts = {k: torch.cat(v).mean(0, keepdim=True) for k, v in style_facts.items()}
#rdn = random.randrange(70,100)
#style_facts['last'] = style_facts['last'] + torch.randint(rdn,size=(1,6,256,16,16)).cuda()
#style_facts['skip'] = style_facts['last'] + torch.rand(1,6,256,16,16).cuda()

### 5. Generate the images.
* `gen_chars`: The characters to generate.
* `save_dir`: Path to save the generated images.
* `source_path`: Path to the font file used as the source font.

In [ ]:
import random
from collections import Counter
import numpy as np
random.seed(100)

In [ ]:
from datetime import datetime
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
def denoise(img):
    # 이진화
    _, binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

    # 노이즈 제거
    kernel = np.ones((3,3), np.uint8)  # 노이즈 제거에 사용할 커널 설정
    closing = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel) # 글자 내부 노이즈 제거
    opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel) # 바깥 노이즈 제거

    # 흰색 배경 제거하고 검정색 글자만 남기기
    mask = closing != 255  # 흰색이 아닌 부분만 True로 설정
    transparent_img = np.zeros((closing.shape[0], closing.shape[1], 4), dtype=np.uint8)  # RGBA 이미지를 위한 빈 Canvas 생성
    transparent_img[mask, 0:3] = 0  # 검정색 글자 설정
    transparent_img[mask, 3] = 255  # 흰색이 아닌 부분은 불투명하게

    # 결과 이미지 보기
    #plt.imshow(transparent_img)
    #plt.axis('off')
    #plt.show()

    #cv2.imwrite('output.png', transparent_img)
    return transparent_img

In [ ]:
########################################################
gen_chars = "아이쿠 화이팅"  # Characters to generate
ref_style = ref_path.split("/")[-1]
checkpoint = (weight_path.split("/")[-1]).split(".")[0]
folder_name = datetime.today().strftime("%Y_%m_%d_%H:%M:%S")
save_dir = Path(f"./results/{checkpoint}_{ref_style}_{folder_name}_{gen_chars}")  # Directory where you want to save generated images
source_path = "/content/drive/MyDrive/2023학교/AIKU/2023_2_project2/source_ttf/gungseo.ttf"  # Path to the font file to render the source images
########################################################


"""# variation
cfg['seed'] = random.random()
cfg['g_args']['experts']['n_experts'] = 6
gen = models.Generator(1, cfg.C, 1, **g_kwargs).cuda().eval()
weight = torch.load(weight_path)
if "generator_ema" in weight:
    weight = weight["generator_ema"]
gen.load_state_dict(weight)"""



save_dir.mkdir(parents=True, exist_ok=True)

source_font = read_font(source_path)
cnt = Counter(gen_chars)
crop_list = []
for char in gen_chars:
    path = save_dir / f"{char}_{cnt[char]}.png"
    if char == ' ' :
        print(char)
        space = np.full((128, 128, 3), 255, dtype = np.uint8)
        im = Image.fromarray(space)
        im.save(path)
    else :
        source_img = transform(render(source_font, char)).unsqueeze(0).cuda()
        char_facts = gen.factorize(gen.encode(source_img), 1)
        rdn = random.randrange(1,85)

        #random_noise = torch.from_numpy(np.random.normal(size=(1,6,256,16,16))).type(torch.FloatTensor)
        #random_noise = torch.from_numpy(np.random.poisson(lam=20, size=(1,6,256,16,16))).type(torch.FloatTensor)
        #random_noise = torch.from_numpy(np.random.uniform(low=0.0, high=rdn, size=(1,6,256,16,16))).type(torch.FloatTensor)
        #char_facts['last'] = char_facts['last'] + random_noise.cuda()
        #char_facts['last'] = char_facts['last'] + torch.randint(rdn,size=(1,6,256,16,16)).cuda()
        #char_facts['skip'] = torch.empty(1, 6, 128, 32, 32).cuda()

        gen_feats = gen.defactorize([style_facts, char_facts])
        out = gen.decode(gen_feats).detach().cpu()[0]
        #pdb.set_trace()
        #out = out.cuda() + torch.rand(1,128,128).cuda()

        cnt[char] -= 1
        save_tensor_to_image(out, path)

    img = cv2.imread(str(path))
    if img is None:
        print(f"ERROR: CANNOT READ {char}.png")
    else:
      tmpimg = img[17:117, 12:116]
      crop_list.append(tmpimg)

CROP_RESULT = cv2.hconcat(crop_list)
cv2.imwrite(str(save_dir / "concat_complete.png"), CROP_RESULT)
concat_img = cv2.imread(str(save_dir / "concat_complete.png"), cv2.IMREAD_GRAYSCALE)  # 그레이스케일로 이미지 불러오기

img_fg = denoise(concat_img)
img_bg = cv2.imread('/content/drive/MyDrive/2023학교/AIKU/2023_2_project2/mxfont-main/background_letter/hobbang.png')

#--② 알파채널을 이용해서 마스크와 역마스크 생성
img_fg = cv2.resize(img_fg, dsize=(700,100),interpolation=cv2.INTER_AREA)
_, mask = cv2.threshold(img_fg[:,:,3], 1, 255, cv2.THRESH_BINARY)
mask_inv = cv2.bitwise_not(mask)

#--③ 전경 영상 크기로 배경 영상에서 ROI 잘라내기
img_fg = cv2.cvtColor(img_fg, cv2.COLOR_BGRA2BGR)
print(img_fg.shape)
h, w = img_fg.shape[:2]
print(img_fg.shape,h,w)
roi = img_bg[70:70+h,10:10+w ]

#--④ 마스크 이용해서 오려내기
print(mask.shape, img_fg.shape)
print(mask_inv.shape, roi.shape)
print(img_bg.shape)
masked_fg = cv2.bitwise_and(img_fg, img_fg, mask=mask)
print(mask_inv.shape)
print(roi.shape)
masked_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)

#--⑥ 이미지 합성
added = masked_fg + masked_bg
img_bg[70:70+h, 10:10+w] = added

cv2.imwrite(str(save_dir / "final_complete.png"), img_bg)


 
(100, 700, 3)
(100, 700, 3) 100 700
(100, 700) (100, 700, 3)
(100, 700) (100, 700, 3)
(599, 800, 3)
(100, 700)
(100, 700, 3)


True